In [1]:
from CLIP import CLIPModel
import torch

/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# checkpoint = torch.load('logs/shared_projector_shared_encoder/best.pt', map_location='cpu')
checkpoint = torch.load('logs/CLIP_PreIVL_PostStent/best.pt', map_location='cpu')
# print(checkpoint.keys())


In [3]:
torch.eq(checkpoint['image_encoder1.model.stages.0.0.convs.0.conv.weight'], checkpoint['image_encoder2.model.stages.0.0.convs.0.conv.weight']).all()

tensor(True)

In [4]:
from nnunetv2.run.run_training import get_trainer_from_args
from nnunetv2.run.load_pretrained_weights import load_pretrained_weights
import config as CFG

trainer = get_trainer_from_args(
    dataset_name_or_id=CFG.nnUNet['dataset_name_or_id'],
    configuration=CFG.nnUNet['configuration'],
    fold=CFG.nnUNet['fold'],
    trainer_name=CFG.nnUNet['trainer_name'],
    plans_identifier=CFG.nnUNet['plans_identifier'],
    device=torch.device('cpu'))
trainer.initialize()

# trainer.load_checkpoint(CFG.nnUNet['checkpoint'])

Using device: cpu


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [5]:
from collections import OrderedDict

encoder_checkpoint = OrderedDict()
for k, v in checkpoint.items():
    if 'image_encoder1.' in k:
        encoder_checkpoint[k.replace('image_encoder1.model.', '')] = v
    
# print(encoder_checkpoint)
load_success = trainer.network.encoder.load_state_dict(encoder_checkpoint)
print(load_success)
# trainer.save_checkpoint('clip_pretrained_nnUNet.pt')
trainer.save_checkpoint('clip_preivl_poststent_pretrained_nnUNet.pt')



<All keys matched successfully>


In [47]:
trainer = get_trainer_from_args(
    dataset_name_or_id=CFG.nnUNet['dataset_name_or_id'],
    configuration=CFG.nnUNet['configuration'],
    fold=CFG.nnUNet['fold'],
    trainer_name=CFG.nnUNet['trainer_name'],
    plans_identifier=CFG.nnUNet['plans_identifier'],
    device=torch.device('cpu'))
trainer.initialize()

load_pretrained_weights(trainer.network, 'clip_pretrained_nnUNet.pt')
for kv_nnUNet, kv_CLIP in zip(trainer.network.encoder.state_dict().items(), encoder_checkpoint.items()):
    assert torch.eq(kv_nnUNet[1], kv_CLIP[1]).all(), f"{kv_nnUNet[0]} and {kv_CLIP[0]} are not equal"


Using device: cpu

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

################### Loading pretrained weights from file  clip_pretrained_nnUNet.pt ###################


/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [51]:
trainer = get_trainer_from_args(
    dataset_name_or_id=CFG.nnUNet['dataset_name_or_id'],
    configuration=CFG.nnUNet['configuration'],
    fold=CFG.nnUNet['fold'],
    trainer_name=CFG.nnUNet['trainer_name'],
    plans_identifier=CFG.nnUNet['plans_identifier'],
    device=torch.device('cpu'))
trainer.initialize()


inequal_keys = []
for kv_nnUNet, kv_CLIP in zip(trainer.network.encoder.state_dict().items(), encoder_checkpoint.items()):
    if not torch.eq(kv_nnUNet[1], kv_CLIP[1]).any()
        inequal_keys.append(kv_nnUNet[0])

print(inequal_keys)
assert len(inequal_keys) > 0, 'all the keys are equal'


Using device: cpu

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


AssertionError: stages.0.0.convs.0.conv.weight and stages.0.0.convs.0.conv.weight are equal

In [46]:
torch.eq(encoder_checkpoint['stages.0.0.convs.0.conv.weight'], trainer.network.encoder.state_dict()['stages.0.0.convs.0.conv.weight']).all()

tensor(True)

In [4]:
from main import make_train_valid_dfs, build_loaders, valid_epoch
import config as CFG


CFG.experiment_name="CLIP_debug"

train_dataframe, valid_dataframe = make_train_valid_dfs()
valid_dl = build_loaders(dataframe=valid_dataframe, mode='valid')

model = CLIPModel()
model.load_state_dict(checkpoint)
model.to(CFG.device)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Using device: cpu

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################



/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


<All keys matched successfully>

In [6]:
from tqdm import tqdm
import torchio as tio

model.eval()

tqdm_object = tqdm(valid_dl, total=len(valid_dl))
for iteration, batch in enumerate(tqdm_object):
    device = CFG.device
    batch['image1'] = batch['image1'][tio.DATA].permute(0, 1, 4, 3, 2).to(device)
    batch['image2'] = batch['image2'][tio.DATA].permute(0, 1, 4, 3, 2).to(device)
    with torch.autocast(device_type="cuda", dtype=torch.float16):
        visualize = iteration % CFG.visualize_every == 0
        loss = model(batch, mode="valid", visualize=visualize)

    count = batch["image1"].size(0)
    break


  0%|          | 0/15 [00:00<?, ?it/s]/home/gridsan/nchutisilp/.local/lib/python3.9/site-packages/torchio/visualization.py:159: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
  0%|          | 0/15 [00:47<?, ?it/s]
